In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. 메모리변수 제거

In [ ]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != '_']
for var in all:
    del globals()[var]

# 2. 사용 패키지

## !pip install --upgrade google-cloud-vision  실행후 런타임 다시시작해야함

In [ ]:
!pip install --upgrade google-cloud-vision 

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, render_template, url_for, request, make_response
from flask_ngrok import run_with_ngrok   # https://ngrok.com/docs
import os
import time
from urllib.parse import quote, unquote

In [ ]:
import re
import io
import os
import cv2
import json
import base64
import pickle
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image
from PIL import Image, ImageDraw, ImageFont
from google.colab import files
from google.colab import drive
from google.cloud import vision
from bs4 import BeautifulSoup
import time
from urllib.request import urlopen
import warnings
import time
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1) # 생략되는 부분 없이 내용 전부 출력

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 3. 데이터로드

## 1) 경로

In [ ]:
path = '/content/drive/MyDrive/Data/데이터셋'

## 2) 음식 이미지 분류 모델

In [ ]:
# 모델 로드
from keras.models import load_model
model = load_model(path + '/kfood/H5/통합_CNN_SAMPLE2.h5')

## 3) 레시피, 재료 데이터

In [ ]:
# 레시피, 재료 데이터 불러오기
df = pd.read_csv(path + '/recipe_en.csv')
df = df.iloc[:, :21]
c_df = pd.read_csv('/content/drive/MyDrive/Data/소스코드/06.practice/플라스크연습/data/crawl_df.csv')

## 4) 리뷰 데이터

In [ ]:
# review 데이터 불러오기
with open(path + '/review_result.pickle', 'rb') as fr:
    review = pickle.load(fr)
# review['후라이드치킨'] = review.pop('후라이드 치킨')
# review['자장면'] = review.pop('짜장면')

## 5) 알레르기 데이터

In [ ]:
# 알레르기 데이터 불러오기
allergy = pd.read_csv(path + '/allergy_Ingredient.csv')
# 읽어올 때 리스트 데이터를 문자열 형태로 읽어온다.
for a in allergy:
    allergy[a] = [eval(allergy[a][0])]

## 6) 메뉴 파인더

In [ ]:
# 메뉴판에서 메뉴 찾기에 쓰이는 메뉴 파인더
menu_finder = pd.read_csv(path + '/menu_finder.csv')

## 7) 키 데이터

In [ ]:
jsonfile = "/content/drive/MyDrive/Data/소스코드/08.GoogleVisionAPI/data/API_KEY.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jsonfile

#4. 출력함수정의

## 1) 레시피 출력 함수

In [ ]:
# 레시피 출력
def get_recipe(food_name):
    recipes = pd.DataFrame(df[df.Small_cat == food_name].iloc[:, 7:21])
    recipes = recipes.transpose()
    recipes = recipes.dropna(axis = 0)
    recipes.columns = [food_name]
    recipes = pd.DataFrame(recipes)
    recipes = recipes.to_dict()
    return recipes

## 2) 알레르기 출력 함수

In [ ]:
# 알레르기 출력
def get_allergy(food_name):
  all = []
  for al in allergy.columns:
      for a in allergy[al][0]:
          for ing in  df[df.Small_cat == food_name].Ingredient.iloc[0].split(', '):
              if a in ing:
                  if al == 'Beef':
                      all.append('Hindu taboo food : ' + ing + '\ntaboo information : ' + al)
                  elif al == 'Pork':
                      all.append('Islam taboo food : ' + ing + '\ntaboo information : ' + al)
                  else:
                      all.append('Allergy caution : '+ ing +'\nAllergy information : ' + al)
  return all

## 3) 리뷰 점수 출력 함수

In [ ]:
# review 점수 결과 반환
def get_review_score(food_name):
    return round(((review[food_name].count('positive') / len(review[food_name])) * 100), 2)

## 4) 음식이름 가지고오기(영어이름 1, 2)

In [ ]:
def get_eng_food_name(food_name):
    eng_name1 = df[df.Small_cat == food_name]['Small_cat_en1'].iloc[0]
    eng_name2 = df[df.Small_cat == food_name]['Small_cat_en2'].iloc[0]
    return eng_name1, eng_name2

## 5) 대표이미지 가지고오기

In [ ]:
def get_representative_image(food_name):
  img_path = 'static/img/repreimg/' + food_name + '.jpg'
  img = Image.open(img_path)
  img_path2 = 'img/repreimg/' + food_name + '.jpg'
  return img_path2

##6) 재료 크롤링

In [ ]:
'''
def get_food_product(food_name):
  ingredient = df[df.Small_cat == food_name].Ingredient.iloc[0]

  ingredient = ingredient.split(',')[:3]
  all_url = []
  all_product = []
  img_list = []
  for ing in ingredient:
    url = f"https://www.ebay.com/sch/i.html?_from=R40&_nkw={ing}&_sacat=14308&LH_TitleDesc=0&rt=nc&LH_BO=1"
    url = url.replace(" ", "%20")
    html = urlopen(url)
    soup = BeautifulSoup(html,'html.parser')
    list = soup.select(".s-item")
    for i in range(1,2):
      img = list[i].select_one(".s-item__image-img")['src']
      urls = list[i].select_one(".s-item__link")['href']
      products = list[i].select_one('.s-item__title').text
      all_url.append(urls)
      all_product.append(products)
      with urlopen(img) as f:
          with open('static/img/crawl/' + ing + str(i)+'.jpg','wb') as h: # 이미지 + 사진번호 + 확장자는 jpg
              img = f.read() #이미지 읽기
              h.write(img) # 이미지 저장
              img_list.append('img/crawl/' + ing + str(i)+'.jpg')
  return ingredient, all_url, all_product, img_list
'''

'\ndef get_food_product(food_name):\n  ingredient = df[df.Small_cat == food_name].Ingredient.iloc[0]\n\n  ingredient = ingredient.split(\',\')[:3]\n  all_url = []\n  all_product = []\n  img_list = []\n  for ing in ingredient:\n    url = f"https://www.ebay.com/sch/i.html?_from=R40&_nkw={ing}&_sacat=14308&LH_TitleDesc=0&rt=nc&LH_BO=1"\n    url = url.replace(" ", "%20")\n    html = urlopen(url)\n    soup = BeautifulSoup(html,\'html.parser\')\n    list = soup.select(".s-item")\n    for i in range(1,2):\n      img = list[i].select_one(".s-item__image-img")[\'src\']\n      urls = list[i].select_one(".s-item__link")[\'href\']\n      products = list[i].select_one(\'.s-item__title\').text\n      all_url.append(urls)\n      all_product.append(products)\n      with urlopen(img) as f:\n          with open(\'static/img/crawl/\' + ing + str(i)+\'.jpg\',\'wb\') as h: # 이미지 + 사진번호 + 확장자는 jpg\n              img = f.read() #이미지 읽기\n              h.write(img) # 이미지 저장\n              img_list.append(\'img

In [ ]:
def get_food_product(food_name): ##크롤링후 데이터 
  ing = eval(c_df[food_name][0])
  url = eval(c_df[food_name][1])
  product = eval(c_df[food_name][2])
  img_list = eval(c_df[food_name][3])
  return ing, url, product, img_list

## 7) 음성인식 클로바

In [ ]:
client_id = 'mu5p1uj3j2'
client_secret = 'rmz4IYQbITxiFn8bjUQ9V9ehr7oDkTo5tOUOJfs3'

lang = "Eng"    # 언어 코드 ( Kor, Jpn, Eng, Chn )
def get_stt(file_path):
  url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
  data = open(file_path, 'rb')
  headers = {
      "X-NCP-APIGW-API-KEY-ID": client_id,
      "X-NCP-APIGW-API-KEY": client_secret,
      "Content-Type": "application/octet-stream"
  }

  response = requests.post(url,  data=data, headers=headers)

  rescode = response.status_code

  #if(rescode == 200):
  #    print(response.text)
  #else:
  #    print("Error : " + response.text)

  return eval(response.text)['text']

## 8) 음식 이미지 BOUNDING BOX, CROP, PREDICT

In [ ]:
def crop_img(file_name):
  client = vision.ImageAnnotatorClient()
  with open(file_name, 'rb') as image_file:
      content = image_file.read()
  image = vision.Image(content = content)
  objects = client.object_localization(image = image).localized_object_annotations
  #print('Number of objects found : {}'.format(len(objects)))
  df = pd.DataFrame(columns = ['name', 'confidence', 'x', 'y'])
  for object_ in objects:
     #print('\n{} (confidence : {})'.format(object_.name, object_.score))
     # print('Normalized bounding polygon vertices : ')
      x, y = [], []
      for vertex in object_.bounding_poly.normalized_vertices:
          #print(' - ({}, {})'.format(vertex.x, vertex.y))
          x.append(vertex.x)
          y.append(vertex.y)
      new_data = {'name' : object_.name, 'confidence' : object_.score, 'x' : x, 'y' : y}
      df = df.append(new_data, ignore_index = True)

  df = df[(df.name == 'Food') | (df.name == 'Bowl')]
  df = df.reset_index().drop(columns ='index')
  return df

def draw_rect(image, point1, point2, label): #사진에 그림그려준다
    font = ImageFont.truetype("LiberationMono-Bold.ttf", 20) #폰트설정 폰트크기설정
    draw = ImageDraw.Draw(image)
    draw.rectangle((point1, point2), outline = (0, 255, 0), width = 2)
    draw.text(point1, label, fill = 'red', font= font)
    return image

def predict_img(img):
  img = img.convert('RGB')
  img = img.resize((64, 64))
  imgarr = np.array(img) / 255
  imgarr = np.array([imgarr,], dtype = 'float32')
  #print(imgarr.shape)
  #print(type(imgarr))
  pred = model.predict(imgarr)
  #print('----- X -----')
  #plt.imshow(imgarr)
  #plt.show()
  class_names = ['갈비구이', '갈비찜', '갈치조림', '계란말이', '고등어구이',
                '김밥', '김치찌개', '닭갈비', '닭볶음탕', '떡볶이',
                '라면', '만두', '미역국', '배추김치', '보쌈',
                '불고기', '비빔밥', '삼겹살', '삼계탕', '양념치킨',
                '육개장', '잡채', '제육볶음', '족발', '자장면',
                '짬뽕', '파전', '후라이드치킨']
#  print('----- PREDICT -----')
#  for i in pred:
#      print(np.argmax(i), class_names[np.argmax(i)])
  food_name = class_names[np.argmax(pred)]
  return food_name

## 9)메뉴판 CLOVA

In [ ]:
# 메뉴판 clova
def get_menu_img_txt(img_path):
    f = open(img_path, 'rb')
    img = base64.b64encode(f.read())
    # 본인의 APIGW Invoke URL로 치환
    URL = "https://753be9f191e84997b6a1cb14d20ae459.apigw.ntruss.com/custom/v1/11837/f75a234e7e7b1d793b15cc0e685df3ff05ba5a55c3702d20de053111d4882541/general"
    # 본인의 Secret Key로 치환
    KEY = "aVVRWGxBZUJkbUhqUXBpd05OaktuUlpPdFVuaU5QQ0M="
    headers = {
        "Content-Type": "application/json",
        "X-OCR-SECRET": KEY
    }
    data = {
        "version": "V1",
        "requestId": "sample_id", # 요청을 구분하기 위한 ID, 사용자가 정의
        "timestamp": 0, # 현재 시간값
        "images": [
                   {"name": "sample_image",
                    "format": "png",
                    "data": img.decode('utf-8')
                    # "templateIds": [400]  # 설정하지 않을 경우, 자동으로 템플릿을 찾음
                    }
                   ]
    }
    data = json.dumps(data)
    response = requests.post(URL, data=data, headers=headers)
    res = json.loads(response.text)
    
    menu_all = res['images'][0]['fields']
    # res['images'][0]['fields']
    # 클로바 ocr에서 찾은 텍스트 중에서 메뉴들만 추출한다.
    found_menu = []
    for i in range(len(menu_finder)):
        for menu in menu_all:
            temp = {}
            for fn in menu_finder['find_name'][i].split(','):
                if fn in menu['inferText']:
                    temp['menu_name'] = menu_finder['menu_name'][i]
                    temp['inferText'] = menu['inferText']
                    temp['data_h'] = menu_finder['data_h'][i]
                    # 좌표가지고 오기
                    temp['xy'] = [menu['boundingPoly']['vertices'][0]['x'], menu['boundingPoly']['vertices'][0]['y'], menu['boundingPoly']['vertices'][2]['x'], menu['boundingPoly']['vertices'][2]['y']]
                    # del(temp['find_name'])
                    found_menu.append(temp)
    return found_menu

#5. 플라스크 실행

In [ ]:
cd /content/drive/MyDrive/Data/소스코드/06.practice/플라스크연습

/content/drive/.shortcut-targets-by-id/1B2PXMquHmFYbjasOH-sMLC0fygfEpZ71/4조/소스코드/06.practice/플라스크연습


In [ ]:
from flask import g
app=Flask(__name__)
run_with_ngrok(app)
global crop_df2
global found_menu

## 메인 홈페이지 접속
@app.route("/", methods=['GET'])
def index_main_loading():
    return render_template("main.html")

## main ==> bts
@app.route('/bts', methods=['GET', 'POST'])
def to_bts():
    if request.method == 'GET':
        return render_template("bts.html")
    else:
        return render_template("main.html")

# bts ==> text 
@app.route('/text', methods=['GET', 'POST'])
def text():
  if request.method == 'GET':
    return render_template("text.html")
  else:
    return render_template("bts.html")

# bts ==> voice
@app.route('/voice', methods=['GET', 'POST'])
def voice():
  if request.method == 'GET':
    return render_template("voice.html")
  else:
    return render_template("bts.html")

# bts ==> photo2
@app.route('/photo', methods=['GET', 'POST'])
def photo():
  if request.method == 'GET':
    return render_template("photo2.html")
  else:
    return render_template("bts.html")


@app.route('/photo2', methods=['GET', 'POST'])
def photo2():
  if request.method == 'GET':
    return render_template("photo.html")
  else:
    return render_template("photo2.html")

@app.route('/photo1', methods=['GET', 'POST'])
def photo1():
  if request.method == 'GET':
    return render_template("photo1.html")
  else:
    return render_template("photo2.html")

@app.route('/menu', methods=['GET', 'POST'])
def menu():
  if request.method == 'GET':
    return render_template("menu.html")
  else:
    return render_template("bts.html")

@app.route('/output_text', methods=['POST']) ## 텍스트 입력시
def output_text():
  if request.method == 'POST': ## if food_name = '': 이면 다른페이지로 넘어가는걸로
    food_name_eng = ''
    food_name_eng = request.form['food_name']
    #if food_name_eng == '':
    if df['Small_cat_en1'].str.lower().isin([food_name_eng.lower()]).sum(): 
      food_name =  df.Small_cat[df['Small_cat_en1'].str.lower().isin([food_name_eng.lower()])].iloc[0]
      eng_name1, eng_name2 = get_eng_food_name(food_name)
      filepath = get_representative_image(food_name)
      allergy = get_allergy(food_name)
      review_score = get_review_score(food_name)
      ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
      tables = get_recipe(food_name)[food_name]
      ingred, url, product, img_list = get_food_product(food_name)
      return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                             ingred=ingred, url=url, product=product, img_list=img_list)

      
    if df['Small_cat_en2'].str.lower().isin([food_name_eng.lower()]).sum(): 
      food_name =  df.Small_cat[df['Small_cat_en2'].str.lower().isin([food_name_eng.lower()])].iloc[0]
      eng_name1, eng_name2 = get_eng_food_name(food_name)
      filepath = get_representative_image(food_name)
      allergy = get_allergy(food_name)
      review_score = get_review_score(food_name)
      ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
      tables = get_recipe(food_name)[food_name]
      ingred, url, product, img_list = get_food_product(food_name)
      return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                             ingred=ingred, url=url, product=product, img_list=img_list)
      
    else:
      return render_template("text.html")

      x, y, z, q = get_food_product(food_name)
      show_food_product(x, y, z, q)
    return render_template("text.html")

@app.route('/ouput_voice', methods=['POST']) ##음성입력시
def output_voice():
  if request.method == 'POST':
    file = request.files['file_voice']
    fname=file.filename
    file.save(os.path.join("static/voice/", fname))
    time.sleep(5)
    f_name_voice = os.path.join("/content/drive/MyDrive/Data/소스코드/06.practice/플라스크연습/static/voice/", fname)

    food=[]
    food_sound_result = get_stt(f_name_voice)
    for i in df['Small_cat_en1']:
      if i.lower() in food_sound_result:
        print(i)
        food.append(i)
    for i in df['Small_cat_en2']:
      if i.lower() in food_sound_result:
        print(i)
        food.append(i)

    if food == []:
      return render_template("voice.html")
    elif df[df['Small_cat_en1']==food[0]]['Small_cat'].sum():
        food_name = df[df['Small_cat_en1']==food[0]]['Small_cat'].iloc[0]
        eng_name1, eng_name2 = get_eng_food_name(food_name)
        filepath = get_representative_image(food_name)
        allergy = get_allergy(food_name)
        review_score = get_review_score(food_name)
        ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
        tables = get_recipe(food_name)[food_name]
        ingred, url, product, img_list = get_food_product(food_name)
        return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                              ingred=ingred, url=url, product=product, img_list=img_list)
    else:
        food_name = df[df['Small_cat_en2']==food[0]]['Small_cat'].iloc[0]
        eng_name1, eng_name2 = get_eng_food_name(food_name)
        filepath = get_representative_image(food_name)
        allergy = get_allergy(food_name)
        review_score = get_review_score(food_name)
        ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
        tables = get_recipe(food_name)[food_name]
        ingred, url, product, img_list = get_food_product(food_name)
        return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                              ingred=ingred, url=url, product=product, img_list=img_list)
  else:
    return render_template("voice.html")

@app.route('/photo_crop', methods=['POST'])
def photo_crop():
  if request.method == 'POST':
    file = request.files['file_photo']
    fname=file.filename

    file.save(os.path.join("static/img/foods_img/", fname))
    file_path = os.path.abspath("static/img/foods_img/"+fname)
    print(file_path)
    crop_df = crop_img(file_path)

    image = Image.open(file_path)
    w, h = image.size
    label_pre =[]
    for i in range(len(crop_df)):
        x = crop_df.x[i]
        y = crop_df.y[i]
        crop_imgs = image.crop((w * min(x), h * min(y), w * max(x), h * max(y)))
        img = crop_imgs
        pre = predict_img(img)
        label_pre.append(pre)
    crop_df['label'] = label_pre

    for i in range(len(crop_df)):
      eng = df[df.Small_cat == crop_df['label'][i]]['Small_cat_en1'].iloc[0] # 영어명으로 넣었다(한글사용시 한글폰트 업데이트 필요)
      label = str(i) + '=' + eng
      x = crop_df.x[i]
      y = crop_df.y[i]
      image = draw_rect(image, (w * min(x), h * min(y)), (w * max(x), h * max(y)), label)
    
    image.save(os.path.join("static/img/crop_food/", fname))
    filepath = "img/crop_food/" + fname
    global crop_df2
    crop_df2 = crop_df.copy()
    return render_template("food_select.html", filepath=filepath)
  else:
     return render_template("photo2.html")

@app.route('/output_photo_select', methods=['POST'])
def output_photo_select():
  if request.method == 'POST':
    global crop_df2
    food_num = request.form['num']
    food_name = crop_df2['label'][int(food_num)]
    eng_name1, eng_name2 = get_eng_food_name(food_name)
    filepath = get_representative_image(food_name)
    allergy = get_allergy(food_name)
    review_score = get_review_score(food_name)
    ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
    tables = get_recipe(food_name)[food_name]
    ingred, url, product, img_list = get_food_product(food_name)
    return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                          ingred=ingred, url=url, product=product, img_list=img_list)
  else:
     return render_template("photo2.html")
@app.route('/photo_result', methods=['POST'])
def photo_result():
  if request.method == 'POST':
    file = request.files['file_photo']
    fname=file.filename
    file.save(os.path.join("static/img/one_food", fname))
    time.sleep(5)
    file_path = os.path.join("/content/drive/MyDrive/Data/소스코드/06.practice/플라스크연습/static/img/one_food/", fname)
    img = Image.open(file_path)
    food_name = predict_img(img)
    eng_name1, eng_name2 = get_eng_food_name(food_name)
    filepath = get_representative_image(food_name)
    allergy = get_allergy(food_name)
    review_score = get_review_score(food_name)
    ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
    tables = get_recipe(food_name)[food_name]
    ingred, url, product, img_list = get_food_product(food_name)
    return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                          ingred=ingred, url=url, product=product, img_list=img_list)
  else:
     return render_template("photo2.html")

@app.route('/menu_output', methods=['POST'])
def menu_output():
  if request.method == 'POST':
    menu_text = []
    file = request.files['file_photo']
    fname=file.filename
    file.save(os.path.join("static/img/menu/", fname))
    time.sleep(5)
    file_path = os.path.abspath("static/img/menu/"+fname)
    found_menu = get_menu_img_txt(file_path)
    img = cv2.imread(file_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # OpenCV는 기본적으로 BGR 형식을 따르므로, RGB 형식으로 변경
    image_width, image_height = Image.open(file_path).size[0], Image.open(file_path).size[1]
    for i, menu in enumerate(found_menu):
        xy = menu['xy']
        # 왼쪽 위 좌표와 오른쪽 아래 좌표
        x1, y1 = xy[0], xy[1]
        x2, y2 = xy[2], xy[3]
        if menu['data_h'] == 1:
            c = (0, 255, 0) # mask인 경우 초록색
        else:
            c = (0, 0, 255)
        # 사각형 그리기
        cv2.rectangle(
            img,
            (int(x1), int(y1)), # 왼쪽 위 좌표
            (int(x2), int(y2)), # 오른쪽 아래 좌표
            color = c, # 색상
          thickness = 4, # 굵기
        )
        cv2.putText(img, str(i) , (int(x1-20), int((y1+y2)/2)), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,0),1, cv2.LINE_AA)
        # cv2.putText(img, "It's Winter", (350, 40), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 1, cv2.LINE_AA)
        
        if menu['data_h'] == 1:
          print(f"{i} = {get_eng_food_name(menu['menu_name'])}")
          menu_text.append(f"{i} = {get_eng_food_name(menu['menu_name'])}")
        else:
          print(f"{i} = no menu data")
          menu_text.append(f"{i} = no menu data")
        # 텍스트 그리기
    cv2.imwrite("static/img/menu_crop/"+fname,img)
    filepath = "img/menu_crop/" + fname
    global found_menu2
    found_menu2 = found_menu
    return render_template("menu_select.html",filepath=filepath, menu_text = menu_text)
  else:
    return render_template("menu.html")

@app.route('/output_menu_select', methods=['POST'])
def output_menu_select():
  if request.method == 'POST':
    global found_menu2
    found_menu2 = [menu for menu in found_menu2 if menu['data_h']==1]
    food_num = request.form['num']
    food_name = found_menu2[int(food_num)]['menu_name']

    eng_name1, eng_name2 = get_eng_food_name(food_name)
    filepath = get_representative_image(food_name)
    allergy = get_allergy(food_name)
    review_score = get_review_score(food_name)
    ing = df[df.Small_cat == food_name].Ingredient.iloc[0]
    tables = get_recipe(food_name)[food_name]
    ingred, url, product, img_list = get_food_product(food_name)
    return render_template("output.html",eng_name1= eng_name1, eng_name2= eng_name2, filepath=filepath, allergy=allergy, review_score=review_score, ing=ing, tables=tables,
                      ingred=ingred, url=url, product=product, img_list=img_list)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://666f-35-230-91-145.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Dec/2021 13:46:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:30] "GET /static/img/배경이미지.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:30] "GET /static/img/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /static/img/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /static/img/배경이미지.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /bts HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /static/img/menu.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:31] "GET /static/img/photo.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:32] "GET /static/img/voice.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:32] "GET /static/img/text.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:46:33] "GET /bts HTT

0 = ('Bulgogi', 'Korean bbq')
1 = ('Bibimbap', 'Bibimbap')
2 = ('Kimchi jjigae', 'Kimchi Stew')
3 = no menu data
4 = no menu data
5 = no menu data
6 = no menu data
7 = no menu data
8 = no menu data


127.0.0.1 - - [07/Dec/2021 13:47:00] "GET /static/img/menu_crop/메뉴판.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:47:20] "POST /output_menu_select HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:47:20] "GET /static/img/repreimg/비빔밥.jpg HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:47:20] "GET /static/img/crawl/%20spinach1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:47:21] "GET /static/img/crawl/%20soy%20bean%20sprouts1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 13:47:21] "GET /static/img/crawl/cooked%20rice1.jpg HTTP/1.1" 200 -
